In [1]:
# ----------------------------
# 0. Install required packages
# ----------------------------
#!pip install tensorflow wandb pillow --quiet

In [23]:
# ----------------------------
# 1. Import libraries
# ----------------------------
import os
import numpy as np
import json
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback
import os
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import wandb
import numpy as np
from PIL import Image
import zipfile


In [24]:
# ----------------------------
# 2. Initialize W&B
# ----------------------------
wandb.login()
wandb.init(project="cats_vs_dogs", name="run_2")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch/accuracy,▁▂▂▃▂▃▃▄▄▄▃▃▄▅▅▆▆▆▆▆▇▆▇▇▇▇█▇█▇█▃▃▅▄▅▅▅▅▅
epoch/epoch,▁▁▁▁▂▂▃▁▁▃▁▁▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇███▁▁▂▂▃▃▃▄▄▄
epoch/learning_rate,██████████████████████████████▂▂▂▂▁▁▁▁▁▁
epoch/loss,▆▆▆▆▆▅▅▅▆▆▆▅▅▅▅▄▃▃▃▃▂▂▂▂▂▁▂▁▂▁▁▇▇█▇▅▄▄▄▄
epoch/val_accuracy,▂▂▄▄▄▆▅▂▃▄▆▅▆▆▇▅▅▅▆▅▇▇▇█▆▇█▇▇▇▁▂▂▃▄▆▆▆▆▆
epoch/val_loss,▃▂▂▂▂▃▃▂▃▃▂▂▂▂▂▃▂▂▂▁▂▁▁▁▂▂▂▃▂▂█▃▇▄▃▃▂▂▂▂
epoch/accuracy,0.726
epoch/epoch,27
epoch/learning_rate,1e-05
epoch/loss,0.54608
epoch/val_accuracy,0.6975


In [ ]:
# ----------------------------
# 3. Download ZIP to working directory
# ----------------------------
work_dir = os.getcwd()
url = "https://zenodo.org/record/5226945/files/cats_dogs_light.zip?download=1"

zip_path = tf.keras.utils.get_file(
    fname="cats_dogs_light.zip",
    origin=url,
    extract=False,
    cache_dir=work_dir,
    cache_subdir=""
)

print(f"ZIP downloaded to: {zip_path}")

# ----------------------------
# 4. Extract ZIP
# ----------------------------
extract_dir = "../data"
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"ZIP extracted to: {extract_dir}")

# copy content from extracted folder to extract_dir and remove extracted folder
extracted_folder = os.path.join(extract_dir, "cats_dogs_light")
for item in os.listdir(extracted_folder):
    s = os.path.join(extracted_folder, item)
    d = os.path.join(extract_dir, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)
shutil.rmtree(extracted_folder)
print(f"Content moved from {extracted_folder} to {extract_dir} and original folder removed.")


# remove the zip file to save space
os.remove(zip_path)

ZIP downloaded to: c:\Users\Besitzer\Documents\FH\Semester 3\SDC\cats_vs_dogs\notebook\cats_dogs_light.zip
ZIP extracted to: ../data
Content moved from ../data\cats_dogs_light to ../data and original folder removed.


In [26]:
# ----------------------------
# 5. Within train and test folders create 'cats' and 'dogs' subfolders and move images accordingly
# ----------------------------

base_dirs = ['train', 'test']
categories = ['cats', 'dogs']

for base_dir in base_dirs:
    base_path = os.path.join(extract_dir, base_dir)
    for category in categories:
        category_path = os.path.join(base_path, category)
        os.makedirs(category_path, exist_ok=True)

    for filename in os.listdir(base_path):
        if filename.startswith('cat') and filename.endswith(('.jpg', '.jpeg', '.png')):
            shutil.move(os.path.join(base_path, filename), os.path.join(base_path, 'cats', filename))
        elif filename.startswith('dog') and filename.endswith(('.jpg', '.jpeg', '.png')):
            shutil.move(os.path.join(base_path, filename), os.path.join(base_path, 'dogs', filename))
print("Images moved to respective category folders.")

Images moved to respective category folders.


In [ ]:
# ----------------------------
# 6. Image generators
# ----------------------------
img_size = (128, 128)
batch_size = 32

train_dir = os.path.join(extract_dir, "train")
test_dir = os.path.join(extract_dir, "test")

# ----------------------------
# Image Data Generators
# ----------------------------
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    "../data/test",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [28]:
print(train_gen.class_indices)  # should show {'0':0, '1':1} or {'cat':0, 'dog':1}
print(val_gen.class_indices)  # should show {'0':0, '1':1} or {'cat':0, 'dog':1}

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


In [ ]:
# ----------------------------
# 7. CNN Model
# ----------------------------
def build_cnn(input_shape=(128,128,3)):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(32, (3,3), activation='relu', padding="same", input_shape=input_shape))
    model.add(layers.MaxPooling2D(2,2))

    # Block 2
    model.add(layers.Conv2D(64, (3,3), activation='relu', padding="same"))
    model.add(layers.MaxPooling2D(2,2))

    # Block 3
    model.add(layers.Conv2D(128, (3,3), activation='relu', padding="same"))
    model.add(layers.MaxPooling2D(2,2))

    # Dense head
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))   # small dropout is enough
    model.add(layers.Dense(1, activation='sigmoid'))

    return model


model = build_cnn()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,287,809 (16.36 MB)

 Trainable params: 4,287,809 (16.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ----------------------------
# 8. Callbacks and Training
# ----------------------------

from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import os

# Directory to save model checkpoints within W&B run
model_dir = wandb.run.dir

callbacks = [
    WandbMetricsLogger(),  # logs metrics automatically
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    # WandbModelCheckpoint(
    #     filepath=os.path.join(model_dir, 'cats_vs_dogs_epoch-{epoch:02d}.keras'),
    #     monitor='val_loss',
    #     verbose=1,
    #     save_best_only=False,  # change to True if you only want best model
    #     save_weights_only=False,
    #     mode='auto',
    #     save_freq='epoch'
    # ),
    EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
]

# Training with generators
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=200,
    callbacks=callbacks
)


Epoch 1/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 603ms/step - accuracy: 0.5450 - loss: 0.7272 - val_accuracy: 0.5000 - val_loss: 0.7037 - learning_rate: 0.0010
Epoch 2/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 250ms/step - accuracy: 0.5450 - loss: 0.6906 - val_accuracy: 0.5000 - val_loss: 0.6935 - learning_rate: 0.0010
Epoch 3/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 232ms/step - accuracy: 0.5450 - loss: 0.6884 - val_accuracy: 0.5000 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 4/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 217ms/step - accuracy: 0.5440 - loss: 0.6907 - val_accuracy: 0.5000 - val_loss: 0.6901 - learning_rate: 0.0010
Epoch 5/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 219ms/step - accuracy: 0.5510 - loss: 0.6843 - val_accuracy: 0.5400 - val_loss: 0.6828 - learning_rate: 0.0010
Epoch 6/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 240ms/step - accuracy: 0.5620 - loss: 0.6790 - val_accuracy: 0.5175 - val_loss: 0.6752 - learning_rate: 0.0010
Epoch 7/200
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 223ms/step - accuracy: 0.6090 - loss: 0

In [ ]:
# ----------------------------
# 9. Save final model
# ----------------------------
model_save_path = os.path.join("../model", "cats_dogs_model.keras")
model.save(model_save_path)
print(f"Final model saved as: {model_save_path}")

Final model saved as: ../model\cats_dogs_model.keras
